In [4]:
library(qvalue)
library(dplyr)
library(ggplot2)

In [2]:
Giordanos_selection = "/hps/nobackup/stegle/users/galvari/data/iPSCs/singleCell/metadata/ensembl_gene/Ensembl_75_Gene_CV_quant5.txt"
G_file = read.csv(Giordanos_selection, sep = "\t")
nrow(G_file)
head(G_file,2)

[1] 21592

,gene,mean,std,CV,quartile_cv
2,ENSG00000227232,3.1805905,3.6232403,1.139172,2
3,ENSG00000243485,0.2003178,0.8649483,4.317881,3


In [3]:
df_common_genes = read.csv("/hps/nobackup/stegle/users/acuomo/all_scripts/sc_eqtl/common_genes_across_all_aggregation_methods_and_bulk.csv")
common_genes = as.character(df_common_genes$gene)
length(common_genes)

[1] 12720

In [5]:
bulk_matched_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/Bulk/BulkDay0Healthy_PCA20_88_206/"

In [6]:
res = read.csv(paste0(bulk_matched_folder,"qtl_results_all.txt"), sep = "\t")
nrow(res)

[1] 16691457

In [7]:
bulk_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/Bulk/BulkTotal_PCA20/"

In [8]:
res_all = read.csv(paste0(bulk_folder,"qtl_results_all.txt"), sep = "\t")
nrow(res_all)

[1] 18974762

In [65]:
leads = read.csv(paste0(bulk_matched_folder,"top_qtl_results_all.txt"), sep = "\t")
nrow(leads)

[1] 43103

In [69]:
df0 = leads
df0 <- df0[-which(duplicated(df0$feature_id)),]
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))
# df1$BH_fdr = p.adjust(df1$empirical_feature_p_value, method = 'BH')
# length(unique(df1[df1$BH_fdr < 0.05,'feature_id']))
df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

[1] 42791

[1] 12720

[1] 2603

In [70]:
max_qval = max(df1[df1$q_value < 0.05,'q_value'])
max_qval
pv_star_matched = max(df1[df1$q_value==max_qval,"p_value"])
pv_star_matched

[1] 0.04992676

[1] 0.00180727

In [71]:
df3 = df1[df1$q_value < 0.05,]
df5 = inner_join(df3, res_all, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df5 <- df5[-which(duplicated(df5$feature_id)),]
nrow(df5)

nrow(df5[(df5$p_value.bulk < pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),])
nrow(df5[df5$p_value.bulk < pv_star_all,])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 2590

[1] 2424

[1] 2424

In [62]:
leads_all = read.csv(paste0(bulk_folder,"top_qtl_results_all.txt"), sep = "\t")
nrow(leads_all)

[1] 49916

In [63]:
df0 = leads_all
df0 <- df0[-which(duplicated(df0$feature_id)),]
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))
df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

[1] 49554

[1] 12720

[1] 9729

In [64]:
max_qval = max(df1[df1$q_value < 0.05,'q_value'])
max_qval
pv_star_all = max(df1[df1$q_value==max_qval, "p_value"])
pv_star_all

[1] 0.04989374

[1] 0.01194398

In [ ]:
##### Mean

In [72]:
mean_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/SingleCell/PseudoBulk/mean/"

In [78]:
my_folder = paste0(mean_folder,"Run_Output_PCA20_88_log_TPM_scater_libsize_206/") # scran, 20PCs
my_folder = paste0(mean_folder,"Run_Output_PCA20_88_log_TPM_scater_libsize_206_baynorm/") # baynorm, 20PCs

In [79]:
df0 = read.csv(paste0(my_folder,"top_qtl_results_all.txt"), sep = "\t")
df0 <- df0[-which(duplicated(df0$feature_id)),]

In [80]:
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))

df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

df3 = df1[df1$q_value < 0.05,]
# nrow(df3)

[1] 50425

[1] 12720

[1] 1860

In [81]:
df4 = inner_join(df3, res, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df4 <- df4[-which(duplicated(df4$feature_id)),]
nrow(df4)

nrow(df4[(df4$p_value.bulk < pv_star_matched & 
         (df4$beta.sc*df4$beta.bulk)>0),])
nrow(df4[(df4$p_value.bulk < pv_star_matched),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1860

[1] 956

[1] 976

In [82]:
df5 = inner_join(df3, res_all, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df5 <- df5[-which(duplicated(df5$feature_id)),]
nrow(df5)

nrow(df5[(df5$p_value.bulk < pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),])
nrow(df5[(df5$p_value.bulk < pv_star_all),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1855

[1] 1347

[1] 1439

In [31]:
##### Total Mean

In [83]:
total_mean_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/SingleCell/PseudoBulk/totmean/"

In [84]:
my_folder = paste0(total_mean_folder,"Run_Output_PCA20_88_log_TPM_scater_libsize_206/") # scran, 20PCs

In [85]:
df0 = read.csv(paste0(my_folder,"top_qtl_results_all.txt"), sep = "\t")
df0 <- df0[-which(duplicated(df0$feature_id)),]

In [86]:
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))
df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

df3 = df1[df1$q_value < 0.05,]
# nrow(df3)

[1] 50429

[1] 12720

[1] 1305

In [87]:
df4 = inner_join(df3, res, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df4 <- df4[-which(duplicated(df4$feature_id)),]
nrow(df4)

nrow(df4[(df4$p_value.bulk < pv_star_matched & 
         (df4$beta.sc*df4$beta.bulk)>0),])
nrow(df4[(df4$p_value.bulk < pv_star_matched),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1305

[1] 788

[1] 808

In [88]:
df5 = inner_join(df3, res_all, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df5 <- df5[-which(duplicated(df5$feature_id)),]
nrow(df5)

nrow(df5[(df5$p_value.bulk < pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),])
nrow(df5[(df5$p_value.bulk < pv_star_all),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1301

[1] 1036

[1] 1108

In [78]:
##### Median

In [89]:
median_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/SingleCell/PseudoBulk/median/"

In [90]:
my_folder = paste0(median_folder,"Run_Output_PCA20_88_log_TPM_scater_libsize_206/") # scran, 20PCs

In [91]:
# df = read.csv(paste0(my_folder,"qtl_results_all.txt"), sep = "\t")
# df <- df[order(df$empirical_feature_p_value, df$p_value ),]
# df0 <- df[-which(duplicated(df$feature_id)),]
# length(unique(df0$feature_id))

In [92]:
df0 = read.csv(paste0(my_folder,"top_qtl_results_all.txt"), sep = "\t")
median_genes = unique(df0$feature_id)
length(median_genes)

[1] 13628

In [93]:
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))
df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

df3 = df1[df1$q_value < 0.05,]
nrow(df3)

df4 = inner_join(df3, res, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df4 <- df4[-which(duplicated(df4$feature_id)),]
nrow(df4)

nrow(df4[(df4$p_value.bulk < pv_star_matched & 
         (df4$beta.sc*df4$beta.bulk)>0),])
nrow(df4[(df4$p_value.bulk < pv_star_matched),])

df5 = inner_join(df3, res_all, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df5 <- df5[-which(duplicated(df5$feature_id)),]
nrow(df5)

nrow(df5[(df5$p_value.bulk < pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),])
nrow(df5[(df5$p_value.bulk < pv_star_all),])

[1] 13628

[1] 12720

[1] 1337

[1] 1340

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

...

[1] 694

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1333

[1] 939

[1] 1019

In [58]:
###### Total Median

In [94]:
totmedian_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/SingleCell/PseudoBulk/totmedian/"

In [95]:
my_folder = paste0(totmedian_folder,"Run_Output_PCA20_88_log_TPM_scater_libsize_206/") # scran, 20PCs

In [96]:
df0 = read.csv(paste0(my_folder,"top_qtl_results_all.txt"), sep = "\t")
df0 <- df0[-which(duplicated(df0$feature_id)),]

In [97]:
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))
df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

df3 = df1[df1$q_value < 0.05,]
nrow(df3)

[1] 12727

[1] 12720

[1] 602

[1] 602

In [98]:
df4 = inner_join(df3, res, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df4 <- df4[-which(duplicated(df4$feature_id)),]
nrow(df4)

nrow(df4[(df4$p_value.bulk < pv_star_matched & 
         (df4$beta.sc*df4$beta.bulk)>0),])
nrow(df4[(df4$p_value.bulk < pv_star_matched),])

df5 = inner_join(df3, res_all, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df5 <- df5[-which(duplicated(df5$feature_id)),]
nrow(df5)

nrow(df5[(df5$p_value.bulk < pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),])
nrow(df5[(df5$p_value.bulk < pv_star_all),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 600

[1] 388

[1] 403

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 599

[1] 470

[1] 518

In [29]:
##### Sum

In [99]:
sum_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/SingleCell/PseudoBulk/sum/"

In [100]:
my_folder = paste0(sum_folder,"Run_Output_PCA20_88_log_TPM_edgeR_libsize_206/") # edgeR, 20PCs

In [101]:
df0 = read.csv(paste0(my_folder,"top_qtl_results_all.txt"), sep = "\t")
df0 <- df0[-which(duplicated(df0$feature_id)),]

In [102]:
length(unique(df0$feature_id))
# df1 = df0[df0$feature_id %in% G_file$gene,]
df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))
df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

df3 = df1[df1$q_value < 0.05,]
nrow(df3)

[1] 50427

[1] 12720

[1] 1463

[1] 1463

In [103]:
df4 = inner_join(df3, res, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df4 <- df4[-which(duplicated(df4$feature_id)),]
nrow(df4)

nrow(df4[(df4$p_value.bulk < pv_star_matched & 
         (df4$beta.sc*df4$beta.bulk)>0),])
nrow(df4[(df4$p_value.bulk < pv_star_matched),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1451

[1] 854

[1] 873

In [104]:
df5 = inner_join(df3, res_all, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df5 <- df5[-which(duplicated(df5$feature_id)),]
nrow(df5)

nrow(df5[(df5$p_value.bulk < pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),])
nrow(df5[(df5$p_value.bulk < pv_star_all),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1448

[1] 1136

[1] 1208

In [ ]:
##### Total Sum

In [89]:
total_sum_folder = "/hps/nobackup/hipsci/scratch/ComparingQtlMapping/SingleCell/PseudoBulk/totsum/"

In [90]:
my_folder = paste0(total_sum_folder,"Run_Output_PCA20_88_log_TPM_edgeR_libsize_206/") # edgeR, 20PCs

In [91]:
# df = read.csv(paste0(my_folder,"qtl_results_all.txt"), sep = "\t")
# df <- df[order(df$empirical_feature_p_value, df$p_value ),]
# df0 <- df[-which(duplicated(df$feature_id)),]
# # nrow(df0)
# length(unique(df0$feature_id))

[1] 50429

In [ ]:
# write.table(df0, "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_eqtl/top_qtl_results_totsum.txt")

In [105]:
df0 = read.csv("/hps/nobackup/stegle/users/acuomo/all_scripts/sc_eqtl/top_qtl_results_totsum.txt", sep = ' ')
head(df0,2)

,feature_id,snp_id,p_value,beta,beta_se,empirical_feature_p_value,feature_chromosome,feature_start,feature_end,n_samples,n_e_samples,alpha_param,beta_param,snp_chromosome,snp_position,assessed_allele,call_rate,maf,hwe_p
1955841,ENSG00000203875,6_86387888_G_T,1.514293e-27,1.010238,0.09289432,6.375394e-24,6,86370710,86388451,87,87,0.7886659,17.12808,6,86387888,T,1,0.4655172,0.6673797
6927785,ENSG00000204650,17_43679121_G_A,9.296484e-23,-1.201981,0.12240973,2.026242e-20,17,43697694,43725582,87,87,0.7567724,29.08926,17,43679121,A,1,0.2701149,0.1020851


In [106]:
length(unique(df0$feature_id))
df1 = df0[df0$feature_id %in% G_file$gene,]
# df1 = df0[df0$feature_id %in% common_genes,]
length(unique(df1$feature_id))
df1$q_value = qvalue(df1$empirical_feature_p_value)$qvalues
length(unique(df1[df1$q_value < 0.05,'feature_id']))

df3 = df1[df1$q_value < 0.05,]
nrow(df3)

[1] 50429

[1] 20545

[1] 1291

[1] 1291

In [107]:
df4 = inner_join(df3, res, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df4 <- df4[-which(duplicated(df4$feature_id)),]
nrow(df4)

nrow(df4[(df4$p_value.bulk < pv_star_matched & 
         (df4$beta.sc*df4$beta.bulk)>0),])
nrow(df4[(df4$p_value.bulk < pv_star_matched),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1283

[1] 808

[1] 828

In [108]:
df5 = inner_join(df3, res_all, by = c("snp_id","feature_id"), suffix = c(".sc",".bulk"))
df5 <- df5[-which(duplicated(df5$feature_id)),]
nrow(df5)

nrow(df5[(df5$p_value.bulk < pv_star_all & 
         (df5$beta.sc*df5$beta.bulk)>0),])
nrow(df5[(df5$p_value.bulk < pv_star_all),])

Warning message:
“Column `snp_id` joining factors with different levels, coercing to character vector”Warning message:
“Column `feature_id` joining factors with different levels, coercing to character vector”

[1] 1286

[1] 1016

[1] 1090